# Notebook de test

Ce notebook qui contient un script de test pour tester localement sa webapp

Dans un terminal, vous pouvez faire `uvicon app:app --reload` pour lancer la webbapp FastAPI qui sert le modèle

Vous pouvez ensuite dérouler / éditer à votre convenance ce notebook pour vérifier que tout se déroule bien

On utilise le package https://requests.readthedocs.io/en/master/ pour effectuer des requêtes

In [6]:
import requests
import io
from PIL import Image
from pathlib import Path
import base64

In [7]:
url = "http://localhost:8000/{path}"

## Test de requêtes de bases

On effectue des requêtes GET sur `/health` et `/models` pour vérifier que tout répond bien

In [9]:
response = requests.get(url.format(path="health"))

assert response.status_code == 200

print("TEST ALIVE PASSED")

TEST ALIVE PASSED


In [11]:
response = requests.get(url.format(path="models"))

assert response.status_code == 200

response = response.json()

assert response == ["yolov5s", "yolov5m", "yolov5l"]

print("TEST MODELS PASSED")

TEST MODELS PASSED


## Requête de processing

On effectue une requête de processing qui est une requête POST sur `/predict` qui contient le nom du modèle ainsi qu'une image à traiter encodée sous la forme de string (on utilise l'encodage base64)

```json
{
    "model":"yolov5s",
    "image":"..."
```

In [24]:
path = Path("cats.jpg")

# ouverture de l'image
image = Image.open(path)

# encodage
with io.BytesIO() as buffer:
    image.save(buffer, format="PNG")
    buffer: str = base64.b64encode(buffer.getvalue()).decode("utf-8")

    # creation de la "payload" de la requête
    data = {"model": "yolov5s", "image": buffer}

    # envoi de la requête
    response = requests.post(url.format(path="predict"), json=data)

assert response.status_code == 200

response = response.json()

assert "time" in response

assert "model" in response

In [25]:
response

{'detections': [{'x_min': 269,
   'y_min': 335,
   'x_max': 793,
   'y_max': 884,
   'class_name': 'cat',
   'confidence': 0.706},
  {'x_min': 1492,
   'y_min': 362,
   'x_max': 2000,
   'y_max': 780,
   'class_name': 'cat',
   'confidence': 0.683},
  {'x_min': 1553,
   'y_min': 676,
   'x_max': 1998,
   'y_max': 1136,
   'class_name': 'cat',
   'confidence': 0.582},
  {'x_min': 753,
   'y_min': 222,
   'x_max': 1085,
   'y_max': 543,
   'class_name': 'cat',
   'confidence': 0.58},
  {'x_min': 0,
   'y_min': 365,
   'x_max': 364,
   'y_max': 910,
   'class_name': 'cat',
   'confidence': 0.575},
  {'x_min': 1449,
   'y_min': 118,
   'x_max': 1787,
   'y_max': 486,
   'class_name': 'cat',
   'confidence': 0.547},
  {'x_min': 644,
   'y_min': 411,
   'x_max': 1190,
   'y_max': 1078,
   'class_name': 'cat',
   'confidence': 0.524}],
 'time': 0.421,
 'model': 'yolov5s'}